## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-11-28-01-15-09 +0000,nyt,Carney Lifts Climate Laws for New Alberta Pipe...,https://www.nytimes.com/2025/11/27/world/canad...
1,2025-11-28-01-14-53 +0000,wapo,Afghan national who once worked with the CIA c...,https://www.washingtonpost.com/nation/2025/11/...
2,2025-11-28-01-08-59 +0000,wapo,Live updates: National Guard member dies after...,https://www.washingtonpost.com/dc-md-va/2025/1...
3,2025-11-28-01-08-59 +0000,wapo,Trump will call Beckstrom’s family; says suspe...,https://www.washingtonpost.com
4,2025-11-28-01-08-56 +0000,bbc,National Guard member dies after shooting in W...,https://www.bbc.com/news/articles/cwyx4y6007lo...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2335/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
8,national,26
16,guard,24
20,trump,22
108,fire,21
104,hong,20


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
18,2025-11-28-00-34-01 +0000,startribune,Trump says one of the two West Virginia Nation...,https://www.startribune.com/afghan-national-in...,128
2,2025-11-28-01-08-59 +0000,wapo,Live updates: National Guard member dies after...,https://www.washingtonpost.com/dc-md-va/2025/1...,114
11,2025-11-28-00-57-00 +0000,nypost,Trump claims National Guard shooting suspect ‘...,https://nypost.com/2025/11/27/us-news/trump-cl...,111
75,2025-11-27-20-14-05 +0000,nypost,Prosecutors will seek death penalty against Af...,https://nypost.com/2025/11/27/us-news/prosecut...,110
238,2025-11-27-04-06-00 +0000,wsj,Trump Calls D.C. National Guard Shooting ‘Act ...,https://www.wsj.com/us-news/national-guard-sho...,101


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
18,128,2025-11-28-00-34-01 +0000,startribune,Trump says one of the two West Virginia Nation...,https://www.startribune.com/afghan-national-in...
113,92,2025-11-27-17-29-01 +0000,nyt,Hong Kong Fire Shared Similarities With Grenfe...,https://www.nytimes.com/2025/11/26/world/asia/...
62,47,2025-11-27-21-18-13 +0000,nyt,Here’s a Look at the D.C. Shooting Suspect’s C...,https://www.nytimes.com/2025/11/27/us/dc-shoot...
245,45,2025-11-27-02-53-50 +0000,nypost,"Hunter, Joe Biden pictured together for first ...",https://nypost.com/2025/11/26/us-news/hunter-j...
246,38,2025-11-27-02-38-44 +0000,nyt,U.K. Budget Plan Calms Markets and Labour Fait...,https://www.nytimes.com/2025/11/26/world/europ...
80,37,2025-11-27-19-31-33 +0000,nypost,National Guard member critically wounded in su...,https://nypost.com/2025/11/27/us-news/national...
133,35,2025-11-27-15-28-14 +0000,cbc,Carney’s energy deal went down very differentl...,https://www.cbc.ca/news/politics/livestory/alb...
29,31,2025-11-27-23-53-10 +0000,nypost,Deli worker in posh NYC nabe arrested for thro...,https://nypost.com/2025/11/27/us-news/deli-wor...
192,30,2025-11-27-10-40-00 +0000,wsj,Silver Prices Rise as China’s Stockpile Hits 1...,https://www.wsj.com/finance/commodities-future...
143,30,2025-11-27-15-02-00 +0000,wsj,Pope Leo Heads to Turkey and Lebanon for His F...,https://www.wsj.com/world/europe/pope-leo-firs...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
